In [15]:
! pip install requests bs4 lxml selenium 

In [16]:
import os
import re
import time
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from selenium import webdriver
from concurrent.futures import ThreadPoolExecutor, as_completed
import json


In [17]:
# PROXIES_JSON_FILE_PATH = 'proxies.json'


In [18]:
# def load_proxies():
#     try:
#         with open(PROXIES_JSON_FILE_PATH, 'r') as file:
#             proxies_data = json.load(file)
#             proxies = [{'ip': proxy['ip'], 'port': proxy['port']}
#                        for proxy in proxies_data]
#             return proxies
#     except FileNotFoundError:
#         print(f"The file {PROXIES_JSON_FILE_PATH} was not found.")
#         return []
#     except json.JSONDecodeError:
#         print("Error decoding the JSON file.")
#         return []


def setup_driver():
    driver = webdriver.Firefox()
    return driver

In [19]:
def fetch_sitemap(url):
    driver = setup_driver()
    driver.get(url)
    time.sleep(10)
    page_source = driver.page_source
    driver.close()
    return page_source


def parse_sitemap(sitemap_content):
    soup = BeautifulSoup(sitemap_content, 'html.parser')
    links = soup.find_all('a')
    urls = [link.get('href') for link in links]
    return urls

In [20]:
def fetch_and_parse(url):
    driver = setup_driver()
    driver.get(url)
    time.sleep(30)
    content = driver.page_source
    content_length = int(len(content) * 0.65)
    driver.close()
    return BeautifulSoup(content[:content_length], 'html.parser')


def is_valid_url(url):
    try:
        result = urlparse(url)
        if result.path.lower().endswith(('.png', '.svg', '.gif')):
            return False
        return all([result.scheme, result.netloc])
    except ValueError:
        return False


def extract_vin_and_images(soup, base_url):
    vin = None
    images = set()

    vin_pattern = re.compile(r'[A-HJ-NPR-Z0-9]{17}')
    for script in soup.find_all('script'):
        if script.string:
            match = vin_pattern.search(script.string)
            if match:
                vin = match.group()
                break
    print(f"VIN: {vin}")
    gallery_div = soup.find('div', id='gallery-modal')
    if gallery_div:
        for img in gallery_div.find_all('img'):
            # Check both 'src' and 'data-src' attributes for image URLs
            src_urls = [img.get(attr)
                        for attr in ['src', 'data-src'] if img.get(attr)]
            for src_url in src_urls:
                if src_url.startswith('/'):
                    src_url = base_url + src_url
                if is_valid_url(src_url):
                    images.add(src_url)
    print(f"Images: {images}")
    return vin, list(images)


def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)


def process_url(url, parsed_url, website_name, data):
    soup = fetch_and_parse(url)
    vin, images = extract_vin_and_images(
        soup, base_url=f"{parsed_url.scheme}://{parsed_url.netloc}"
    )

    if vin:
        data['vins'][vin] = {'scraped_images_url': images}

    with open(os.path.join(website_name, 'inventory_car_data.json'), 'w') as f:
        json.dump(data, f, indent=4)


def filter_urls(urls, pattern):
    return [url for url in urls if pattern in url]

In [21]:
def main(url):

    parsed_url = urlparse(url)
    website_name = parsed_url.netloc
    # check for other sitemap urls
    sitemap_url = f'https://{website_name}/dealer-inspire-inventory/inventory_sitemap'

    create_directory(website_name)
    try:
        sitemap_content = fetch_sitemap(sitemap_url)
        urls = parse_sitemap(sitemap_content)
        inventory_urls = filter_urls(urls, '/inventory')
        with open(os.path.join(website_name, 'inventory_car_urls.txt'), 'w') as f:
            f.writelines(", \n".join(inventory_urls))

        data = {
            'website_url': website_name,
            'vins': {}
        }

        with ThreadPoolExecutor(max_workers=1) as executor:
            futures = []
            for inventory_url in inventory_urls:
                futures.append(executor.submit(
                    process_url, inventory_url, parsed_url, website_name, data))
            for future in as_completed(futures):
                result = future.result()

    except Exception as e:
        print("An error occurred:", str(e))


if __name__ == "__main__":

    website_input_url = 'https://www.airportcdj.com/inventory/new-2023-dodge-charger-sxt-rwd-sedan-2c3cdxbg9ph646474/'
    main(website_input_url)

An error occurred: Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.airportcdj.com/dealer-inspire-inventory/inventory_sitemap&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20www.airportcdj.com.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:832:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:58:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:330:39
emit@resource://gre/modules/EventEmitter.sys.mjs:148:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:33:25

